In [23]:
import pyodbc
import pandas as pd
import sqlite3
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [24]:


DB_SOURCE = {
    'servername': r'LAPTOP-PRQNR6SS\SQLEXPRESS',
    'database': 'united_outdoors_sdm'
}


# DB_TARGET = {
#     'servername': r'LAPTOP-PRQNR6SS\SQLEXPRESS',
#     'database': '????'
# }

# Maak verbindingen
source_conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    f"SERVER={DB_SOURCE['servername']};"
    f"DATABASE={DB_SOURCE['database']};"
    "Trusted_Connection=yes;"
)

# target_conn = pyodbc.connect(
#     'DRIVER={SQL Server};'
#     f"SERVER={DB_TARGET['servername']};"
#     f"DATABASE={DB_TARGET['database']};"
#     "Trusted_Connection=yes;"
# )

# Cursors maken
source_cursor = source_conn.cursor()
# target_cursor = target_conn.cursor()

print("Succesvol verbonden met beide databases!")

Succesvol verbonden met beide databases!


In [ ]:
# Haal een lijst van tabellen op uit de bron-database
sql_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"


# Gebruik pandas om de query uit te voeren en de tabellen op te halen
tables_df = pd.read_sql(sql_query, source_conn)

# Print de lijst van tabellen
tables_df

#### AW order

In [21]:
salesorder_query = """
SELECT h.id AS order_id, h.customer_id, h.order_date, SUM(d.line_total) AS total_amount
FROM salesorder_header_aw h
JOIN salesorder_detail_aw d ON h.id = d.order_id
GROUP BY h.id, h.customer_id, h.order_date
"""

salesorder_aw_df = pd.read_sql(salesorder_query, source_conn)

salesorder_aw_df



,order_id,customer_id,order_date,total_amount
0,43659,29825,2021-05-31,20565.6206
1,43660,29672,2021-05-31,1294.2529
2,43661,29734,2021-05-31,32726.4786
3,43662,29994,2021-05-31,28832.5289
4,43663,29565,2021-05-31,419.4589
...,...,...,...,...
31460,75119,11981,2024-06-30,42.2800
31461,75120,18749,2024-06-30,84.9600
31462,75121,15251,2024-06-30,74.9800
31463,75122,15868,2024-06-30,30.9700


#### NW order


In [18]:
order_query_nw = """
SELECT h.id AS order_id, h.customer_id, h.order_date, SUM(d.unit_price * d.quantity) AS total_amount
FROM order_nw h
JOIN order_details_nw d ON h.id = d.order_id
GROUP BY h.id, h.customer_id, h.order_date
"""

order_nw_df = pd.read_sql(order_query_nw, source_conn)

order_nw_df


,order_id,customer_id,order_date,total_amount
0,10248,VINET,2022-07-04,440.0
1,10249,TOMSP,2022-07-05,1863.4
2,10250,HANAR,2022-07-08,1813.0
3,10251,VICTE,2022-07-08,670.8
4,10252,SUPRD,2022-07-09,3730.0
...,...,...,...,...
825,11073,PERIC,2024-05-05,300.0
826,11074,SIMOB,2024-05-06,244.3
827,11075,RICSU,2024-05-06,586.0
828,11076,BONAP,2024-05-06,1057.0


####  AW employee

In [33]:
# Query voor hr_employee_aw
hr_employee_aw_query = "SELECT business_entity_id AS id, gender FROM hr_employee_aw"
hr_employee_aw_df = pd.read_sql(hr_employee_aw_query, source_conn)

# Query voor person_aw
person_aw_query = "SELECT business_entity_id, first_name, middle_name, last_name FROM person_aw"
person_aw_df = pd.read_sql(person_aw_query, source_conn)

# Merge beide tabellen op business_entity_id
employee_aw_merge = pd.merge(hr_employee_aw_df, person_aw_df, left_on="id", right_on="business_entity_id", how="inner")

# Optioneel: business_entity_id kolom verwijderen als je die niet nodig hebt
employee_aw_merge.drop(columns=["business_entity_id", "id"], inplace=True)

# Resultaat tonen
employee_aw_merge

,gender,first_name,middle_name,last_name
0,M,Ken,J,Sánchez
1,F,Terri,Lee,Duffy
2,M,Roberto,None,Tamburello
3,M,Rob,None,Walters
4,F,Gail,A,Erickson
...,...,...,...,...
285,F,Lynn,N,Tsoflias
286,F,Amy,E,Alberts
287,F,Rachel,B,Valdez
288,F,Jae,B,Pak
